In [5]:
from rdkit import Chem, Geometry
from crest_confs_utils import (get_low_energy_conformer,
                               xtbError,
                               display_3d_mol,
                               xtb_single_point)

In [2]:
import subprocess
import tempfile

In [15]:
def crest_conf_search(input_mol: Chem.rdchem.Mol,
                 charge: int = 0,
                 e_state: int = 0,
                 solvent: str = '',
                 mmff_max_iters: int = 200) -> Chem.rdchem.Mol:
    # makes sure that there is a conformer embedded in mol
    mol = Chem.rdchem.Mol(input_mol)
    if len(mol.GetConformers()) == 0:
        mol = get_low_energy_conformer(mol, mmff_max_iters)

    xtb_xyz = ''
    # runs calculations in tmp directory
    with tempfile.TemporaryDirectory() as tmp:
        # create .xyz file in the tmp directory
        Chem.rdmolfiles.MolToXYZFile(mol, f'{tmp}/input.xyz')
        # run xtb on the input file
        xtb_args = ['-c', str(charge), '-u', str(e_state)]
        if solvent != '':
            xtb_args += ['-g', solvent]
        proc = subprocess.run(['crest', 'input.xyz'] + xtb_args, 
                              cwd=tmp,
                            #   stdout=subprocess.PIPE,
                              stdout=None, 
                              stderr=subprocess.DEVNULL)
        if proc.returncode != 0:
            raise xtbError('xtb abnormal termination')
        # with open(f'{tmp}/xtbopt.xyz') as file:
        #     # first two lines of xyz are atom count and comments
        #     # last line is blank
        #     xtb_xyz = file.read().split('\n')[2:len(xtb_xyz)-1]

    # # creates a new RDKit Mol with embedded conformer from the xtb xyz output
    # mol.RemoveAllConformers()
    # conf = Chem.rdchem.Conformer(mol.GetNumAtoms())
    # for i, line in enumerate(xtb_xyz):
    #     ls = line.split()
    #     x, y, z = float(ls[1]), float(ls[2]), float(ls[3])
    #     conf.SetAtomPosition(i, Geometry.rdGeometry.Point3D(x, y, z))
    # mol.AddConformer(conf)
    # return mol

In [16]:
# m = Chem.MolFromSmiles('OC(CN1CCN(C2=C(F)C=NC=C2F)CC1)=O')
m = Chem.MolFromSmiles('OCCO')
m = crest_conf_search(m)



       |                                            |
       |                 C R E S T                  |
       |                                            |
       |  Conformer-Rotamer Ensemble Sampling Tool  |
       |          based on the GFN methods          |
       |             P.Pracht, S.Grimme             |
       |          Universitaet Bonn, MCTC           |
       Version 2.11.2, Fr 17. Dec 12:10:44 CEST 2021
  Using the xTB program. Compatible with xTB version 6.4.0

   Cite work conducted with this code as

   P. Pracht, F. Bohle, S. Grimme, PCCP, 2020, 22, 7169-7192.

   and  S. Grimme, JCTC, 2019, 15, 2847-2862.

   with help from:
   C.Bannwarth, F.Bohle, S.Ehlert, S.Grimme,
   C. Plett, P.Pracht, S. Spicher

   This program is distributed in the hope that it will be useful,
   but WITHOUT ANY WARRANTY; without even the implied warranty of
   MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

 Command line input:
 > crest input.xyz -c 0 -u 0


 --------------